## Jeu de données
Un labo avec des projets, des services et des personnes

In [1]:
import json 
with open('data/lab2000.json','r') as f:
    lab = json.load(f)

# Liste des projets
lab['projects']

['Alpha', 'Cymbal', 'Tigre', 'Csw2000', 'Gringalet']

In [2]:
# Liste des services
lab['services']

['Electronique', 'Informatique', 'Optique', 'Mécanique']

In [3]:
# Liste des personnes (les 3 premières)
print(json.dumps(lab['persons'][1:3], indent=4, sort_keys=True))

[
    {
        "name": "Arlette Bienvenue",
        "projects": [
            {
                "function": "developpeur",
                "name": "Alpha"
            }
        ],
        "service": "Electronique"
    },
    {
        "name": "Roxanne Brochu",
        "projects": [],
        "service": "Informatique"
    }
]


# Neo4j - Python

## Jouons avec le driver officiel neo4j python

Le documentation est ici (version 1.7)

https://neo4j.com/docs/api/python-driver/1.7-preview/

## Concepts de base

- Un driver gère toutes les interactions avec la base
- Ces interactions se déroulent dans le cadre d'une session fournie par le driver
- les modifications se déroule au sein d'une transaction fournie par la session

```
driver        session        transaction
 |               |               |               
 | session()     |               |
 |-------------->|               |
 |               |   begin()     |               
 |               |-------------->|               
 |               |               |               
 |               |               |----+            
 |               |               |   write something
 |               |               |<---+           
 |               |               |               
 |               |               |----+           
 |               |               |  commit()     
 |               |               |<---+           
 |               |               |               
 |               |               |----+
 |               |               | close()              
 |               |               |<---+               
 |               |               |               
```



## Interactions avec la base
* Instancier un driver
* Obtenir une session

In [4]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver('bolt://localhost:7687')

## Modification
* Ouverture d'une transaction pour modifier
 * modification
 * commit ou rollback

### Création des noeuds

D'une façon conventionelle, puis d'une façon plus pythonique.

In [5]:
def create_nodes_classic_way(label, data, driver):
    
    # -- Obtention session --
    session = driver.session()
    
    # -- Obention transaction --
    tx = session.begin_transaction()
    
    for name in data:
        tx.run('CREATE (:'+label+' {name : {name}})', name=name)
    
    #-- commit --
    tx.commit()
    # -- fermeture de la transaction. Elle ne peut être réutilisée.
    tx.close()

In [6]:
create_nodes_classic_way('Project', lab['projects'], driver)

In [8]:
def create_nodes_pythonic_way(label, data, driver):
    
    # - mode auto-commit
    
    with driver.session() as local_session:
        for name in data:
            print(name)
            # -- pas de transaction, directement au niveau de la session
            local_session.run('CREATE (:'+label+' {name : {name}})', name=name)

In [9]:
create_nodes_pythonic_way('Service', lab['services'],driver)

Electronique
Informatique
Optique
Mécanique


### Lecture seule

Un simple `Session.run(query)` suffit.

In [10]:
# - session globale pour le notebook
global_session = driver.session()

Les résultats sont des `Records` à parcourir.

In [11]:
[r for r in global_session.run('MATCH (n) RETURN labels(n)[0], n.name')]

[<Record labels(n)[0]='Project' n.name='Alpha'>,
 <Record labels(n)[0]='Project' n.name='Cymbal'>,
 <Record labels(n)[0]='Project' n.name='Tigre'>,
 <Record labels(n)[0]='Project' n.name='Csw2000'>,
 <Record labels(n)[0]='Project' n.name='Gringalet'>,
 <Record labels(n)[0]='Service' n.name='Electronique'>,
 <Record labels(n)[0]='Service' n.name='Informatique'>,
 <Record labels(n)[0]='Service' n.name='Optique'>,
 <Record labels(n)[0]='Service' n.name='Mécanique'>]

In [12]:
global_session.run('MATCH (n:Project) RETURN count(n) as nb').single().value()

5

## Résultats en tant qu'entités Graph
Pour obtenir les résultats sous forme de 
* Node
* Relationship
* Path

il faut utiliser la vue `Graph` des `StatementResult`.

Au sein de graphes, il est possible d'avoir les noeuds `.nodes` et les relations `.relationships`.


In [13]:
[r for r in global_session.run('MATCH (n:Project) RETURN n').graph().nodes]

[<Node id=44 labels={'Project'} properties={'name': 'Alpha'}>,
 <Node id=45 labels={'Project'} properties={'name': 'Cymbal'}>,
 <Node id=46 labels={'Project'} properties={'name': 'Tigre'}>,
 <Node id=47 labels={'Project'} properties={'name': 'Csw2000'}>,
 <Node id=48 labels={'Project'} properties={'name': 'Gringalet'}>]

### Création de relations

**(:Person)-\[:MEMBER_OF\]->(:Service)**

In [14]:
query ='''
MATCH (s:Service {name: {service}}) 
CREATE (p:Person {name: {name}})
CREATE (p)-[:MEMBER_OF]->(s)'''

with driver.session() as session:
    for p in lab['persons']:
        session.run(query, name=p['name'],service=p['service'])

**(:Person)-\[:WORK_IN \{function=$function}\]->(:Project)**

In [15]:
subquery = '''
MATCH (pers:Person {name: {name}}),(project:Project {name: {project}})
MERGE (pers)-[:WORK_IN{function: {function}} ]->(project)'''
        
with driver.session() as session:
    for person in lab['persons']:        
        for project in person['projects']:
            session.run(subquery,
                       name=person['name'],
                       project=project['name'],
                       function=project['function'])

In [16]:
with driver.session() as session:
    records = session.run('MATCH (n)-[r:WORK_IN]->(p) return n,r,p')
    
# - les 5 premiers
for relation in list(records.graph().relationships)[0:5]:
    print(relation.items())
    print(relation.type)
    print(relation.start_node)
    print(relation.end_node)
    print('---')

dict_items([('function', 'expert')])
WORK_IN
<Node id=17 labels={'Person'} properties={'name': 'Leone Bienvenue'}>
<Node id=44 labels={'Project'} properties={'name': 'Alpha'}>
---
dict_items([('function', 'utilisateur')])
WORK_IN
<Node id=16 labels={'Person'} properties={'name': 'Édith Audet'}>
<Node id=44 labels={'Project'} properties={'name': 'Alpha'}>
---
dict_items([('function', 'expert')])
WORK_IN
<Node id=14 labels={'Person'} properties={'name': 'Mireille Audet'}>
<Node id=44 labels={'Project'} properties={'name': 'Alpha'}>
---
dict_items([('function', 'utilisateur')])
WORK_IN
<Node id=13 labels={'Person'} properties={'name': 'Andrée Laforest'}>
<Node id=44 labels={'Project'} properties={'name': 'Alpha'}>
---
dict_items([('function', 'chefDeProjet')])
WORK_IN
<Node id=9 labels={'Person'} properties={'name': 'Marphisa Brochu'}>
<Node id=44 labels={'Project'} properties={'name': 'Alpha'}>
---


### La liste des personnes qui ne sont pas associées à un projet

In [17]:
with driver.session() as session:
    query= '''MATCH (p:Person)-[:MEMBER_OF]->(s) 
    WHERE NOT (p)-[:WORK_IN]->(:Project) 
    RETURN p.name AS name, s.name AS service'''
    records = session.run(query)
    
print(f'Statement {records.summary().statement} after {records.summary().result_available_after} ms' )

[ f"Person : {r['name']}, Service: {r['service']}" for r in records]


Statement MATCH (p:Person)-[:MEMBER_OF]->(s) 
    WHERE NOT (p)-[:WORK_IN]->(:Project) 
    RETURN p.name AS name, s.name AS service after 1 ms


['Person : Lotye Cousteau, Service: Informatique',
 'Person : Carine Laforest, Service: Informatique',
 'Person : Matilda Audet, Service: Mécanique',
 'Person : Inès Cousteau, Service: Informatique',
 'Person : Agathe Desnoyers, Service: Electronique',
 'Person : Valentine Brochu, Service: Electronique',
 'Person : Roxanne Brochu, Service: Informatique']

# Avec py2no

py2neo est un wrapper autour du driver neo4j.

Il simplifie la vie et s'intégre très bien avec plusieurs librairies communes Python

In [18]:
from py2neo import Graph, Node, Relationship

graph = Graph()

### Avec Pandas

In [19]:
query= '''MATCH (p:Person)-[:MEMBER_OF]->(s) 
WHERE NOT (p)-[:WORK_IN]->(:Project) 
RETURN p.name AS name, s.name AS service'''

graph.run(query).to_data_frame()

,name,service
0,Lotye Cousteau,Informatique
1,Carine Laforest,Informatique
2,Matilda Audet,Mécanique
3,Inès Cousteau,Informatique
4,Agathe Desnoyers,Electronique
5,Valentine Brochu,Electronique
6,Roxanne Brochu,Informatique


### Avec Numpy

In [20]:
graph.run(query).to_ndarray()

array([['Lotye Cousteau', 'Informatique'],
       ['Carine Laforest', 'Informatique'],
       ['Matilda Audet', 'Mécanique'],
       ['Inès Cousteau', 'Informatique'],
       ['Agathe Desnoyers', 'Electronique'],
       ['Valentine Brochu', 'Electronique'],
       ['Roxanne Brochu', 'Informatique']], dtype='<U16')

## Manipulations Graphiques

py2neo permet de manipuler le graphe directement en terme de Node, Relationship.

Le graphe est modifié localement avant de mettre à jour la base à travers des _push_ et des _pull_.

In [21]:
# - Création de la ville d'orsay
city = Node('City', name='Orsay')

# - merge en matchant sur le label 'City' et la clef 'name'
graph.merge(city, 'City','name')

In [22]:
# - recherche du noeud (roxane)
roxanne = graph.nodes.match("Person", name='Roxanne Brochu').first()

print(roxanne, type(roxanne))

# - mise en relation avec la ville
liveIn = Relationship(roxanne, "lIVE_IN", city)

# - CREATION de la relation
graph.create(liveIn)


(_57:Person {name: 'Roxanne Brochu'}) <class 'py2neo.data.Node'>


In [23]:
# - Recherche du service informatique

serviceInfo = graph.nodes.match('Service', name='Informatique').first()

# - recherche de la relation entre carine et le service informatique

roxanneToInformatique = graph.match((roxanne,), r_type='MEMBER_OF').first()

# - Mise à jour locale

roxanneToInformatique['function'] = 'chef'

# - mise à jour remote 
graph.push(roxanneToInformatique)

In [24]:
graph.match((roxanne,), r_type='MEMBER_OF').first()

(Roxanne Brochu)-[:MEMBER_OF {function: 'chef'}]->(Informatique)

## Mettre à jour le graphe du projet  'Tigre'

In [38]:
import random

# - récupération duprojet 'tigre'
tigerProject = graph.nodes.match('Project', name='Tigre').first()

# - récupération des relations (p)->[r:WORK_IN]->(:Project {name: 'Tigre'})
relations =list(graph.relationships.match((None,tigerProject),'WORK_IN')) 

# - Mise à jour d'une charge de travail aléatoire comme en vrai ? :)
for rel in relations:
    rel['workload'] = random.randint(1,8)*10


for rel in graph.relationships.match((None,tigerProject),'WORK_IN'):
    print(rel.start_node, rel['workload'])

(_14:Person {name: 'Mireille Audet'}) 20
(_6:Person {name: 'Val\u00e9rie Laforest'}) 10
(_1:Person {name: 'Josette Laforest'}) 30
(_63:Person {name: 'Cosette Audet'}) 60
(_62:Person {name: 'Melodie Dubois'}) 50
(_61:Person {name: 'Darlene Herman'}) 30
(_60:Person {name: 'Heloise Dubois'}) 40


### Regroupement des objet sous forme de Subgraph (conteneur)

In [39]:
from py2neo import Subgraph

subgraph=Subgraph(relationships=relations)
graph.push(subgraph)

In [ ]:
for rel in graph.relationships.match((None,tigerProject),'WORK_IN'):
    print(rel.start_node, rel['workload'])

## Procédure de nettoyage


In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

checkbox = widgets.ToggleButton(
    value=False,
    description='DELETE ALL',
    disabled=False
)

def delete_all(b):
    if b:
        with driver.session() as session:
            session.run('MATCH (n) DETACH DELETE n')
    
            
interact(delete_all, b=checkbox)